# This is an implementation for computing Collision Free Regions in Configuration Space using IRIS

[IRIS in Configuration Space](https://drake.mit.edu/doxygen_cxx/group__geometry__optimization.html#ga3a51e0fec449a0abcf498f78a2a390a8)


This notebook contains code for creating, visualizing and saving such regions.

In [1]:
import os.path
import sys
import numpy as np

from pydrake.geometry import ( StartMeshcat )
from pydrake.geometry.optimization import ( SaveIrisRegionsYamlFile)

# add parent folders to path
sys.path.append(os.path.abspath('..'))

# Update the import
from src.manipulation import running_as_notebook
from src.auxiliar.auxiliar_functions import (VisualizeConnectivity)
from src.auxiliar.homemade_gcs import GcsTrajOpt
from src.auxiliar.iris_options import create_iris_options
from src.auxiliar.joint_teleop import JointTeleop
from src.auxiliar.helper import VisualizeRegions

In [2]:
# Start the visualizer (only run this once).
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


# Setup



In [3]:
# Get the absolute path of the current working directory
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
data_folder = os.path.join(parent_dir, 'data')

iris_filename = os.path.join(data_folder, 'my_iris.yaml')  # The file to save the regions to

iris_regions = dict() # The regions that we will compute

q = []

iris_options, config = create_iris_options() # Create the options for the iris algorithm

## Run IRIS on manually-specified seeds

In [4]:
print(config["iris_regions"].keys())
iris_regions = config["iris_regions"]
VisualizeRegions(meshcat, iris_regions, config)

dict_keys([])


## Joint Teleop

In [5]:
# clear the meshcat visualizer
meshcat.Delete()
meshcat.DeleteAddedControls()

seeds_joint_teleop = []
JointTeleop(meshcat, seeds_joint_teleop, config["iris_regions"], config, q)

INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/43a7e91dde0f8193ccbbdc6a0e49faf7a1aaeefc.tar.gz


RuntimeError: //home/harel/github/gcs-motion-planning/common-files/models/KINOVA_GEN3_LITE/model.sdf:8: error: URI 'package://drake_models/KINOVA_GEN3_LITE/meshes/base_link.obj' resolved to '/home/harel/.cache/drake/package_map/fc4a2817d9f26ef119da8d5ae994af472a5a13edddd62a75bb5373fb4bf3a4cc-cf3a1ff5ed0155a6c84509dcb9ff2f786fd2e3eb0f109ca828574ebf9b1d0118/KINOVA_GEN3_LITE/meshes/base_link.obj' which does not exist.

## Visualize the current IRIS region connectivity

For planning trajectories, it's important to have some non-empty intersection between the convex sets. Here is a simple utility the visualization the connectivity as a graph.

In [ ]:
VisualizeConnectivity(config['iris_regions'])

In [ ]:
print(seeds_joint_teleop)

## Plan trajectories with GCS Trajectory Optimization

In [ ]:
# clear the meshcat visualizer
meshcat.Delete()
meshcat.DeleteAddedControls()

assert (
    seeds_joint_teleop
), "The examples here use the 'manually-specified seeds' from the section above. Please run that section first, or populate your own start and end configurations."

GcsTrajOpt(seeds_joint_teleop[0], seeds_joint_teleop[1],config["iris_regions"], meshcat)

# Save your regions back to disk

Note that by default, this notebook will autosave a backup after making each region.

In [ ]:
if running_as_notebook:
    SaveIrisRegionsYamlFile(iris_filename, iris_regions)